In [2]:

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder \
            .master("local[3]") \
            .appName("MiscDemo") \
            .config("spark.executor.memory", "2g") \
            .getOrCreate()

In [4]:
data_list = [("Ravi", "28", "1", "2002"),
             ("Abdul", "23", "5", "81"), # 1981
             ("John", "12", "12", "6"), # 2006
             ("Rosy", "7", "8", "63"), # 1963
             ("Abdul", "23", "5", "81") # 1981
            ]

In [5]:
raw_df = spark.createDataFrame(data_list).toDF("name","day","month","year").repartition(3)

In [6]:
new_df = raw_df.withColumn("id",monotonically_increasing_id())


inline casting

In [7]:
df2 = new_df.withColumn("year",expr("""
                                    case when year < 21 then cast (year as int ) + 2000 
                                    when year < 100  then cast (year as int ) + 1900
                                    else year
                                    end
                                   """ ))

change the schema

In [ ]:
df2 = new_df.withColumn("year",expr("""
                                    case when year < 21 then cast (year as int ) + 2000 
                                    when year < 100  then cast (year as int ) + 1900
                                    else year
                                    end
                                   """ ).cast(IntegerType()))

case expression

In [ ]:
df2 = new_df.withColumn("year", when(col("year")<21 ,col("year")+2000) \
                        .when(col("year")<100,col("year"+1900).otherwise(col("year"))))

In [ ]:
df8 = new_df.withColumn("dob", expr("to_date(concat(day,'/',month,'/',year), 'd/M/y')"))
df8.show()

In [ ]:
df9 = new_df.withColumn("dob", to_date(expr("concat(day,'/',month,'/',year)"), 'd/M/y')) \
         .drop("day", "month", "year") \
         .dropDuplicates(["name", "dob"]) \
         .sort(expr("dob desc"))
df9.show()